In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from wmfdata import charting, mariadb, hive
from wmfdata.utils import pd_display_all, list_wikis

charting.set_mpl_style()

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


In [ ]:
MEDIAWIKI_HISTORY_SNAPSHOT = "2018-08"

# Surviving translations by newcomers

In [ ]:
hive.run("""
select

from event.mediawiki_revision_tags_change rtc
left join wmf.mediawiki_history mh
on
    rtc.wiki_db = mh.wiki_db and
    rtc.page_id = mh.page_id and
where
    snapshot = "{snapshot}" and
    event_entity = "page"


group by mh.page_id
""")

# Page history data quality

In [40]:
hive.run("""
select
    event_type,
    event_timestamp,
    event_user_text,
    page_title,
    page_title_historical,
    page_id
from wmf.mediawiki_history
where
    event_entity = "page" and
    wiki_db = "enwiki" and
    (page_title_historical = "Northwest_Post-Grunge" or page_title = "Northwest_Post-Grunge") and
    snapshot = "2018-08"
""")

,event_type,event_timestamp,event_user_text,page_title,page_title_historical,page_id
0,create,2007-03-08 10:17:31.0,Jogersbot,Northwest_Post-Grunge,Northwest_Post-Grunge,9930619
1,restore,2018-08-31 23:30:48.0,Explicit,Northwest_Post-Grunge,Northwest_Post-Grunge,7882004
2,restore,2018-08-31 23:30:57.0,Explicit,Northwest_Post-Grunge,Northwest_Post-Grunge,9930619
3,create,2006-11-11 04:16:06.0,Bri,Northwest_Post-Grunge,Northwest_Post-Grunge,7882004


In [34]:
hive.run("""
select
    event_type,
    event_timestamp,
    event_user_text,
    page_title,
    page_title_historical,
    page_id
from wmf.mediawiki_history
where
    event_entity = "page" and
    wiki_db = "enwiki" and
    (page_title_historical = "Jeff_Caldwell_(soccer)" or page_title = "Jeff_Caldwell_(soccer)") and
    snapshot = "2018-08"
""")

,event_type,event_timestamp,event_user_text,page_title,page_title_historical,page_id
0,create,2018-07-19 13:00:57.0,Freefalling660,Freefalling660/Jeff_Caldwell_(soccer),Jeff_Caldwell_(soccer),57939448
1,restore,2018-07-31 17:33:57.0,Hut 8.5,Freefalling660/Jeff_Caldwell_(soccer),Jeff_Caldwell_(soccer),57939448


In [44]:
print(_34)

  event_type        event_timestamp event_user_text  \
0     create  2018-07-19 13:00:57.0  Freefalling660   
1    restore  2018-07-31 17:33:57.0         Hut 8.5   

                              page_title   page_title_historical   page_id  
0  Freefalling660/Jeff_Caldwell_(soccer)  Jeff_Caldwell_(soccer)  57939448  
1  Freefalling660/Jeff_Caldwell_(soccer)  Jeff_Caldwell_(soccer)  57939448  


In [43]:
hive.run("""
select
    page_id,
    page_id_artificial,
    page_title,
    page_title_historical,
    start_timestamp,
    end_timestamp,
    caused_by_event_type,
    caused_by_user_id
from wmf.mediawiki_page_history
where
    wiki_db = "enwiki" and
    (page_title_historical = "Jeff_Caldwell_(soccer)" or page_title = "Jeff_Caldwell_(soccer)") and
    snapshot = "2018-08"
order by start_timestamp asc
limit 1000
""")

,page_id,page_id_artificial,page_title,page_title_historical,start_timestamp,end_timestamp,caused_by_event_type,caused_by_user_id
0,50257013,52fcca51-05a3-4190-b8dd-e640eb83d6f8,Jeff_Caldwell_(soccer),Jeff_Caldwell_(soccer),None,2016-05-03 11:57:22.0,create,NaN
1,50256784,566f8bbd-ce52-44be-bf3a-65d2cacc8111,Jeff_Caldwell_(soccer),Jeff_Caldwell_(soccer),None,2016-05-03 11:57:22.0,create,NaN
2,57939448,eeb6329b-5067-43c1-a9ca-47443565533c,Jeff_Caldwell_(soccer),Freefalling660/Jeff_Caldwell,None,2018-07-20 20:08:30.0,create,NaN
3,50256784,566f8bbd-ce52-44be-bf3a-65d2cacc8111,Jeff_Caldwell_(soccer),Jeff_Caldwell_(soccer),2016-05-03 11:57:22.0,2016-05-03 11:57:22.0,delete,1925080.0
4,57427746,d0f5cd01-65ea-4d76-b8bb-df88afd3f60e,Jeff_Caldwell_(soccer),Jeff_Caldwell_(soccer),2016-05-03 11:57:22.0,2018-05-16 12:13:51.0,create,NaN
5,57427914,36c5d506-499c-4173-b848-0a5d463e18f2,Jeff_Caldwell_(soccer),Jeff_Caldwell_(soccer),2016-05-03 11:57:22.0,2018-05-16 12:13:55.0,create,NaN
6,50257013,52fcca51-05a3-4190-b8dd-e640eb83d6f8,Jeff_Caldwell_(soccer),Jeff_Caldwell_(soccer),2016-05-03 11:57:22.0,2016-05-03 11:57:22.0,delete,1925080.0
7,57427746,d0f5cd01-65ea-4d76-b8bb-df88afd3f60e,Jeff_Caldwell_(soccer),Jeff_Caldwell_(soccer),2018-05-16 12:13:51.0,2018-05-16 12:13:51.0,delete,119438.0
8,57427914,36c5d506-499c-4173-b848-0a5d463e18f2,Jeff_Caldwell_(soccer),Jeff_Caldwell_(soccer),2018-05-16 12:13:55.0,2018-05-16 12:13:55.0,delete,119438.0
9,57978189,57490893-6926-4ac5-85ac-355c9657208c,Jeff_Caldwell_(soccer),Jeff_Caldwell_(soccer),2018-05-16 12:13:55.0,2018-07-30 20:41:30.0,create,NaN


In [45]:
print(_43)

     page_id                    page_id_artificial  \
0   50257013  52fcca51-05a3-4190-b8dd-e640eb83d6f8   
1   50256784  566f8bbd-ce52-44be-bf3a-65d2cacc8111   
2   57939448  eeb6329b-5067-43c1-a9ca-47443565533c   
3   50256784  566f8bbd-ce52-44be-bf3a-65d2cacc8111   
4   57427746  d0f5cd01-65ea-4d76-b8bb-df88afd3f60e   
5   57427914  36c5d506-499c-4173-b848-0a5d463e18f2   
6   50257013  52fcca51-05a3-4190-b8dd-e640eb83d6f8   
7   57427746  d0f5cd01-65ea-4d76-b8bb-df88afd3f60e   
8   57427914  36c5d506-499c-4173-b848-0a5d463e18f2   
9   57978189  57490893-6926-4ac5-85ac-355c9657208c   
10  57939448                                  None   
11  57939448  eeb6329b-5067-43c1-a9ca-47443565533c   
12  57939448  eeb6329b-5067-43c1-a9ca-47443565533c   
13  57978189  57490893-6926-4ac5-85ac-355c9657208c   
14  57939448                                  None   

                               page_title         page_title_historical  \
0                  Jeff_Caldwell_(soccer)        Jeff_Caldwe

In [37]:
hive.run("""
select
    event_type,
    event_timestamp,
    event_user_text,
    page_id
from wmf.mediawiki_history
where
    event_entity = "page" and
    wiki_db = "eswiki" and
    (page_title_historical = "Accidente ferroviario de Cerrillos de 1956" or page_title = "Accidente ferroviario de Cerrillos de 1956") and
    snapshot = "2018-08"
""")

,event_type,event_timestamp,event_user_text,page_id


In [38]:
hive.run("""
select
    page_id,
    page_id_artificial,
    start_timestamp,
    end_timestamp,
    caused_by_event_type,
    caused_by_user_id
from wmf.mediawiki_page_history
where
    wiki_db = "eswiki" and
    page_title = "Accidente ferroviario de Cerrillos de 1956" and
    snapshot = "2018-08"

""")

,page_id,page_id_artificial,start_timestamp,end_timestamp,caused_by_event_type,caused_by_user_id
